In [107]:
# adapted from https://www.geeksforgeeks.org/python-fetch-your-gmail-emails-from-a-particular-user/
# I am only getting the initial email sent, I do not know how this will work with forwarded emails or reply chains

In [136]:
import imaplib, email 
import pandas as pd
import datetime as dt

In [108]:
###!!! DO NOT UPLOAD WITH THIS !!!###

## DO NOT UPLOAD TO GITHUB WITH THIS EITHER ##
user  = "USER_NAME"
password = "PASSWORD"

In [15]:


imap_url = 'imap.gmail.com'
    
# Function to search for a key value pair  
def search(key, value, con):  
    result, data = con.search(None, key, '"{}"'.format(value)) 
    return data 
  
# Function to get the list of emails under this label 
def get_emails(result_bytes): 
    msgs = [] # all the email data are pushed inside an array 
    for num in result_bytes[0].split(): 
        typ, data = con.fetch(num, '(RFC822)') 
        msgs.append(data) 
  
    return msgs 
  
# this is done to make SSL connnection with GMAIL 
con = imaplib.IMAP4_SSL(imap_url)  
  
# logging the user in 
con.login(user, password)  
  
# calling function to check for email under this label 
con.select('Inbox')  

 # fetching emails from this user
msgs = get_emails(search('SUBJECT', 'MOOD', con)) 
  

In [106]:
# msgs is a list of all emails
body = str(msgs[0][0][1], 'utf-8')

# get the start of our 'boundary' tag, so we can identify the body of the email
l_bound = body.find('; boundary="') + 12 # add 12 because that's len('; boundary="')

# get the end of our 'boundary' tag
r_bound = body.find('"', l_bound)

# get the ID of where our body starts and stops
body_id = body[l_bound:r_bound]

# split our email on the body_id, and get the content (2nd element)
# FYI: 2nd element is plain text and 3rd element is HTML
print(body.split(body_id)[2])


Content-Type: text/plain; charset="UTF-8"

9-12-19: 07.24
Valence: 7
Arousal: 7
Inspiration: 8

--


In [202]:
message_contents = []
# printing them by date received 
for i, msg in enumerate(msgs):  
    for content in msg: 
        if type(content) is tuple:
  
            # Handling errors related to unicodenecode 
            try:  
                full_email = str(content[1], 'utf-8')
            except UnicodeEncodeError as e: 
                pass
            
    # get the start of our 'boundary' tag, so we can identify the body of the email
    l_bound = full_email.find('; boundary="') + 12 # add 12 because that's len('; boundary="')

    # get the end of our 'boundary' tag
    r_bound = full_email.find('"', l_bound)

    # get the ID of where our body starts and stops
    body_id = full_email[l_bound:r_bound]

    # split our email on the body_id, and get the content (2nd element)
    # FYI: 2nd element is plain text and 3rd element is HTML
    try:
        body = full_email.split(body_id)[2]
    except:
        body = "Email could not be read: {0}".format(i)        
    message_contents.append(body)

In [205]:
raw_dat = []

for content in message_contents:
    # check to make sure we didn't have an error when reading the email
    # len("Email could not be read: ") = 25
    if "Email could not be read: " == content[:25]:
        raw_dat.append([content, None, None, None, None])
        continue
    
    # split the body based on our new lines
    body_split = content.replace("\r", "").splitlines()

    # get our date and time that we sent this
    # take the third line, which is our date
    date_time_raw = body_split[3]

    # turn our date and time strings into a standard date time object
    date_time = dt.datetime.strptime(date_time_raw, "%m-%d-%y: %H.%M")

    # turn our date_time object into an easily understood string
    date_time = date_time.strftime("%Y/%m/%d %H:%M")

    # get our valance, arousal, and inspiration
    # go through each line of our email
    val = None
    aro = None
    ins = None
    note = None
    for lines in body_split:
        # find our key/val delimeter (": "), it will be -1 if it is missing
        if lines.find(": ") == -1:
            continue
        
        else:
            key, value = lines.split(": ")
            if key == "Valence":
                val = value
            
            elif key == "Arousal":
                aro = value
                
            elif key == "Inspiration":
                ins = value
                
            elif key == "Note":
                note = value

    raw_dat.append([date_time, val, aro, ins, note])
    
emotions = pd.DataFrame(columns=["Date", "Valence", "Arousal", "Inspiration"])

In [206]:
raw_dat

[['2019/09/12 07:24', '7', '7', '8', None],
 ['2019/09/12 09:43', '8', '9', '6', None],
 ['2019/09/12 12:08', '6', '5', '4', None],
 ['2019/09/12 22:26', '5', '2', '1', None],
 ['2019/09/13 06:45', '3', '5', '0', None],
 ['2019/09/13 23:08', '4', '1', '2', None],
 ['2019/09/14 09:00', '5', '4', '0', None],
 ['2019/09/14 13:18', '2', '2', '0', None],
 ['2019/09/14 16:06', '8', '7', '1', None],
 ['2019/09/15 00:24', '8', '2', '0', None],
 ['2019/09/15 08:37', '6', '3', '3', None],
 ['2019/09/15 13:12', '0', '7', '1', None],
 ['2019/09/15 23:23', '4', '2', '0', None],
 ['2019/09/16 07:41', '5', '4', '5', None],
 ['2019/09/16 14:06', '6', '5', '5', None],
 ['2019/09/16 22:36', '6', '3', '1', None],
 ['2019/09/17 16:27', '5', '2', '1', None],
 ['2019/09/17 23:06', '4', '4', '4', None],
 ['2019/09/18 06:39', '2', '3', '0', None],
 ['2019/09/18 22:34', '9', '2', '1', None],
 ['2019/09/19 08:52', '3', '3', '0', None],
 ['2019/09/20 07:08', '3', '5', '1', None],
 ['2019/09/20 22:54', '6', '1', 